# Generate tiled region (aka non-TMA) run file

This automatically creates tiles which define FOVs on the MIBI slide based on user-set parameters for non-TMAs.

Notes about MIBI coordinate axes:

* The x-axis goes from left to right ascending
* The y-axis goes from bottom to top ascending

In [ ]:
import json
import os

from ark.mibi import tiling_utils

### Define paths to the JSON data

Define the following parameters to set your home directory:

* `base_dir`: the root directory of the tiling data
* `json_tiling_dir`: the directory in `base_dir` containing the information to read and write the FOV info

Define the prefix to use for your input and output file names:

* `tiled_region_prefix`: defaults to `example_tiled_region`

Define the following input and output paths (all prefixed by `fov_gen_prefix`):

* `fov_list_path`: the list of FOVs to use for tiling. Each FOV in `fov_list_path` defines the top-left corner of a separate region
* `fov_output_path`: where to store the run file defining the FOVs. Will be an actual run file for non-TMAs and a basic mapping of FOV to centroid coordinate (in microns) for TMAs.
* `moly_path`: the path to the Moly point, needed if you want to insert this between FOVs and/or regions after remapping. Ignored if you choose not to insert Moly points between FOVs.

In [ ]:
# define your home directory
base_dir = "../data/example_dataset"
json_tiling_dir = os.path.join(base_dir, "json_tiling_data")

# define the prefix of each file
tiled_region_prefix = 'example_tiled_region'

# define the input and output files
fov_list_path = os.path.join(json_tiling_dir, '%s_fov_list.json' % tiled_region_prefix)
fov_output_path = os.path.join(json_tiling_dir, '%s_fov_output.json' % tiled_region_prefix)
moly_path = os.path.join(json_tiling_dir, '%s_moly_point.json' % tiled_region_prefix)

### Set tiling parameters

The following parameters need to be defined for each region (you will be prompted for them running this cell):

* The FOV defining the top-left corner and its centroid (taken from `fov_list_path`, each entry indicates a different region)
* Number of FOVs along the x- and y-axis
* x- and y-axis step size
* Whether to randomize the order of the FOVs or not
* Whether to insert Moly points between regions
* Whether to insert Moly points between FOVs at a specified interval. Please refer to the documentation for `tiled_region_generate_fov_list` for how the Moly point insertion works: https://ark-analysis.readthedocs.io/en/latest/_markdown/ark.mibi.html#ark.mibi.tiling_utils.tiled_region_generate_fov_list

In [ ]:
tiling_params, moly_point = tiling_utils.tiled_region_set_params(
    fov_list_path,
    moly_path
)

### Create the FOVs for each region

Tiling will be done from top to bottom (descending y-axis order), then left to right (ascending x-axis order). FOV names correspond to their row and column position on the grid of FOVs and are 1-indexed.

For example, `R1C1` means the FOV is in row 1 and column 1 of the grid of FOVs, `R2C1` would be the next FOV down, and `R1C2` would be the next FOV to the right. In an unrandomized `auto_fov_regions`, `R2C1` would immediately follow `R1C1` and all of the `R{n}C1` FOVs would come before the `R{n}C2` FOVs for each region.

Note that due to the y-axis ascending from bottom to top, `R2C1's` y-axis coordinate is lower compared to `R1C1's`.

In [ ]:
fov_regions = tiling_utils.tiled_region_generate_fov_list(
    tiling_params,
    moly_point
)

In [ ]:
# write the tiled output
with open(fov_output_path, 'w') as fop:
    json.dump(fov_regions, fop)